### 1.リンクを作成

- まず、https://www.jtuc-rengo.or.jp/soudan/qa/ を基盤のリンクとします。このページを開くと、クリック可能な44個の質問があります。
- 44個のリンクから、それぞれのページを一つ一つ開きたい為、 
'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_01.html' -> 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_02.html', ... .https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_44.html を文字列として生成していきます。

In [2]:
# Define a function to count the number of digits in a number
def count_digits(number):
    # Convert the number to a string and count the characters
    num_str = str(number)
    num_digits = len(num_str)
    return num_digits

# Define the number of rows
rows = 44

# Base URL for the links
base_html_link = 'https://www.jtuc-rengo.or.jp/soudan/qa/data/'

# Create an empty list to store the generated links
lst_link = []

# Loop through the range of rows
for idx in range(rows):
    # Check if the index plus one has a single digit or more
    if count_digits(idx + 1) == 1:
        # If it's a single digit, create a link with 'QA_0' followed by the index
        new_link = base_html_link + 'QA_0' + str(idx + 1) + '.html'
    else:
        # If it's more than one digit, create a link with 'QA_' followed by the index
        new_link = base_html_link + 'QA_' + str(idx + 1) + '.html'
    
    # Add the new link to the list
    lst_link.append(new_link)

# Print the list of generated links
print(lst_link)

['https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_01.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_02.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_03.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_04.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_05.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_06.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_07.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_08.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_09.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_10.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_11.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_12.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_13.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_14.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_15.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_16.html', 'https://www.jtuc-rengo.or.jp/soudan/qa/data/QA_17.html', 'https://www.

### 2. 要素を分けて、文字データを取得
- 次に、１ページづつ開き、"タイトル", "質問", "回答"の部分を注目します。 <br> この部分をBeautiful Soup に認識させる為には, htmlコードのclassを特定して、find()関数を利用できます。認識後、それぞれの文字データを取得します。
- データ取得後、jsonファイルに読み込みます。

In [4]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re, json  

specify_link = lst_link[0]

req = Request(specify_link)
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

def scrap_to_csv(lst_link,fileName_json): 

    # create a new csv file 
    fieldnames  = ["タイトル", "質問", "回答", "法律ポイント"] # Define the column headers

    RowsToScrap = 44 # number of rows to scrap
    data_list = []  # stores every rows

    for page_index in range(RowsToScrap): 
        
        # get whole page 
        each_link = lst_link[page_index]
        req = Request(each_link)
        html_page = urlopen(req)
        soup = BeautifulSoup(html_page, "lxml")

        # get title 
        title_element = soup.find("h5", class_="title004")
        title_text = title_element.get_text(strip=True)
        
        # get question 
        div = soup.find("div", {"class": "boxActivityKizunaSentence"})
        q_element = div.find("dt", text="Q")
        q_question_text = q_element.find_next("dd").text.strip()
        
        # get answer
        q_element = div.find("dt", text="A")
        q_answer_text = q_element.find_next("dd").text.strip()

        # get law point
        LawPointText = div.select_one("div.boxEmphasis012.l-mb40 dd")
        LawPointText = str(LawPointText)
        start_index , end_index= LawPointText.find("<dd>"), LawPointText.find("</dd>")  
        LawPointText = LawPointText[start_index+4:end_index]# remove dd tags 

        # Write the new row of data
        new_row = {"タイトル": title_text, "質問": q_question_text, "回答": q_answer_text, "法律ポイント": LawPointText}
        data_list.append(new_row)
        

    # # write data_list to json file 
    with open(fileName_json, "w", encoding="utf-8") as json_file:
        json.dump(data_list, json_file, indent=4, ensure_ascii=False)

    
fileName_json = 'jtuc-rengo-datasource.json'
scrap_to_csv(lst_link,fileName_json)

C:\Users\liewg\AppData\Local\Temp\ipykernel_18096\3658744482.py:34: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  q_element = div.find("dt", text="Q")
C:\Users\liewg\AppData\Local\Temp\ipykernel_18096\3658744482.py:38: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  q_element = div.find("dt", text="A")
